In [20]:
import os
from sys import platform

import torchaudio

from tortoise.api import TextToSpeech
from tortoise.utils.audio import load_voice


ROOT = os.getcwd()

modelDir = os.path.join(ROOT, "models")
outputDir = os.path.join(ROOT, "output")
outputFiles = [file for file in os.listdir(outputDir) if ".DS_Store" not in file]
modelFiles = [file for file in os.listdir(modelDir) if ".DS_Store" not in file]
newModels = [
    file for file in modelFiles if file.replace("_gpt.pth", ".wav") not in outputFiles
]

In [ ]:
for model in newModels:
    model_path = f"models/{model}"
    autoregressive_batch_size = 8 if platform == "darwin" else None
    tts = TextToSpeech(
        model_path=model_path,
        use_deepspeed=True,
        kv_cache=True,
        autoregressive_batch_size=autoregressive_batch_size,
    )

    text = "You know, liberalism used to be something my dad would get into fights over in bars."

    preset = "fast"

    voice = "john"

    voice_samples, conditioning_latents = load_voice(voice)
    gen = tts.tts_with_preset(
        text,
        voice_samples=voice_samples,
        conditioning_latents=conditioning_latents,
        preset=preset,
    )
    torchaudio.save(
        f"output/{model.replace('_gpt.pth', '.wav')}", gen.squeeze(0).cpu(), 24000
    )
    del tts

In [24]:
import IPython.display as ipd

ipd.Audio("output/21000.wav")